<a href="https://colab.research.google.com/github/MK-1729/AI-Geeks-Tutorials/blob/master/MK-1729/Training_Conv_Nets/CustomDatasets/CustomDataset_Pytorch_googlenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os import listdir
from os.path import isfile, join
from os import walk
import os 
import pandas as pd

#onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
# create dataframe
df = pd.DataFrame(columns = ['img','label'])
index=0
root="/content/drive/MyDrive/Leaves_Dataset/D1/train"
for path, subdirs, files in os.walk(root):    
    x=path.split('/')
    if len(x)==7: continue
    for name in files:
        path_dir=os.path.join(x[7], name)
        k=name.split(' ')
        k=k[0].split('Class')
        df.loc[index]=[path_dir,k[1]]
        index=index+1
df

,img,label
0,Class (16)/R_0Class16 (3).jpg,16
1,Class (16)/R_0Class16 (5).jpg,16
2,Class (16)/R_0Class16 (1).jpg,16
3,Class (16)/R_0Class16 (4).jpg,16
4,Class (16)/R_0Class16 (2).jpg,16
...,...,...
2283,Class (23)/R_270Class23 (6).jpg,23
2284,Class (23)/R_270Class23 (76).jpg,23
2285,Class (23)/R_270Class23 (9).jpg,23
2286,Class (23)/R_315Class23 (12).jpg,23


In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.transforms as transforms 
from torch.utils.data import random_split
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim 
from skimage import io

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

In [ ]:
class LeavesDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = csv_file
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)


dataset = LeavesDataset(
    csv_file=df,
    root_dir=root,
    transform=transforms.ToTensor(),
)    
batch_size=100
train_set, test_set = random_split(dataset,[2000,288])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [ ]:
 # Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Cost at epoch 0 is 2.3314852595329283
Cost at epoch 1 is 0.3565385572612286
Cost at epoch 2 is 0.09452374819666147
Cost at epoch 3 is 0.050683731539174916
Cost at epoch 4 is 0.03422307022847235
Cost at epoch 5 is 0.03661580216139555
Cost at epoch 6 is 0.051416009990498425
Cost at epoch 7 is 0.052148801693692806
Cost at epoch 8 is 0.05490411017090082
Cost at epoch 9 is 0.027390565164387225
Checking accuracy on Training Set
Got 1998 / 2000 with accuracy 99.90
Checking accuracy on Test Set
Got 277 / 288 with accuracy 96.18
